# Установка необходимого и настройка окружающей среды

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install wandb
!pip install changefinder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=216d84c9550b3dc032d54a23a4945c6de5c37095f09024f43c11585dab69a8d7
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 2.6 MB/s eta 0:00:00
  Created wheel for changef

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!cp /content/gdrive/MyDrive/cpd/cpd_nn.zip /content

In [ ]:
!unzip /content/cpd_nn.zip -d /content

Archive:  /content/cpd_nn.zip
   creating: /content/cpd_nn/__pycache__/
  inflating: /content/cpd_nn/__pycache__/dataset.cpython-38.pyc  
  inflating: /content/cpd_nn/__pycache__/model.cpython-38.pyc  
  inflating: /content/cpd_nn/__pycache__/train.cpython-38.pyc  
  inflating: /content/cpd_nn/dataset.py  
  inflating: /content/cpd_nn/model.py  
  inflating: /content/cpd_nn/train.py  


In [ ]:
!cp /content/gdrive/MyDrive/cpd/synth_ts.zip /content

In [ ]:
!unzip /content/synth_ts.zip -d /content

Archive:  /content/synth_ts.zip
   creating: /content/synth_ts/__pycache__/
  inflating: /content/synth_ts/__pycache__/synth_timeserie_generator.cpython-38.pyc  
  inflating: /content/synth_ts/synth_timeserie_generator.py  


In [ ]:
!cp /content/gdrive/MyDrive/cpd/timeserie.zip /content

In [ ]:
!unzip /content/timeserie.zip -d /content

Archive:  /content/timeserie.zip
  inflating: /content/timeserie/002.csv  
  inflating: /content/timeserie/timeseire_004.npy  
  inflating: /content/timeserie/timeserie_000.npy  
  inflating: /content/timeserie/timeserie_001.npy  
  inflating: /content/timeserie/timeserie_002.npy  
  inflating: /content/timeserie/timeserie_003.npy  


In [ ]:
import torch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

# Настройка WANDB

In [ ]:
import wandb
import os

os.environ['WANDB_KEY'] = '3dbf6bc6ee9d845aeb9d49a5be8ef71e8c9fc466'
os.environ['WANDB_NOTEBOOK_NAME'] = 'cpd-synth'

# Запуск экспериментов

Параметры

In [ ]:
# dataset_params = {
#     'data_path': '/content/timeserie/timeseire_004.npy',
#     'lambda': 150,
#     'alpha': 0.625,
#     'step_size': 0.07,
#     'min_stability_duration': 1,
#     'window_size': 64,
#     'train_size': 0.7,
#     'horizon': 1,
#     'feature_type': 'stationary',
#     'balancing': 'None',
#     'pos_ratio': 0.5,
#     'lag': 1,
#     'objective': 'regression',
#     'cooldown': 30,
#     'is_round': False,
#     'round_decimals': 0,
# }

dataset_params = {
    'data_path': '/content/timeserie/timeserie_002.npy',
    'lambda': 1.25,
    'alpha': 1.05,
    'step_size': 0.07,
    'min_stability_duration': 1,
    'window_size': 64,
    'train_size': 0.7,
    'horizon': 1,
    'feature_type': 'stationary',
    'balancing': 'None',
    'pos_ratio': 0.5,
    'lag': 1,
    'objective': 'regression',
    'cooldown': 30,
    'is_round': False,
    'round_decimals': 0,
}

MODEL = 'LSTM'
lstm_model_params = {
    'lstm': {
        'input_dim': 1,
        'hidden_dim': 256,
        'num_layers': 3,
        'dropout': 0.000001,
        'is_bidirectional': False,
        'nonlinearity': 'relu'
    },
    'linear': {
        'hidden_dim': 128 
    }
}
# mlp_model_params = {
#     'hidden_dim': 512,
#     'n_hidden': 3,
# }
experiment_params = {
    'lr': 1e-3,
    'batch_size': 6144,
    'num_epochs': 6
    }

In [ ]:
tags = ['gru', 'regression', 'stationary', 'pink']
notes = "4mil timeserie, with values: (x_t - x_{t-1})/x_{t-1} experiment."

In [ ]:
import numpy as np
from cpd_nn.dataset import SynthDatasetTimeToCp, SynthDatasetClassification
from cpd_nn.model import CPD_MLP
from cpd_nn.train import train
import wandb
import torch

with open(dataset_params['data_path'], 'rb') as f:
        ts = np.load(f)
# sd = SynthDatasetClassification(
#     ts, dataset_params['min_stability_duration'], 
#     dataset_params['window_size'],
#     dataset_params['train_size'], 
#     dataset_params['horizon'],
#     dataset_params['feature_type'],
#     dataset_params['balancing'],
#     dataset_params['pos_ratio'],
#     lap=dataset_params['lag'],
#     is_round=dataset_params['is_round'],
#     round_decimals=dataset_params['round_decimals']
# )

sd = SynthDatasetTimeToCp(
    ts, 
    dataset_params['min_stability_duration'],
    dataset_params['window_size'],
    dataset_params['train_size'],
    dataset_params['feature_type'],
    dataset_params['balancing'],
    dataset_params['pos_ratio'],
    dataset_params['cooldown']
)

  0%|          | 0/3999904 [00:00<?, ?it/s]

In [ ]:
train_loader = sd.get_trainloader(  
    batch_size=experiment_params['batch_size'],
    shuffle_train=True,
    model_type = MODEL
)
val_loader = sd.get_testloader(  
    batch_size=experiment_params['batch_size'],
    model_type = MODEL
)

In [ ]:
cnt = 0
for i in range(len(train_loader.dataset)):
  cnt += train_loader.dataset[i][1][0].item()

In [ ]:
cnt

1378568762.0

In [ ]:
print(len(train_loader.dataset))

2649236


In [ ]:
# model = CPD_MLP(input_dim=dataset_params['window_size'], hidden_dim=_model_params['hidden_dim']).to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters(), lr=experiment_params['lr'])
# loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
# from cpd_nn.model import CPD_GRU
# model = CPD_GRU(lstm_model_params['lstm'], lstm_model_params['linear']).to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters(), lr=experiment_params['lr'])
# loss_fn = torch.nn.BCEWithLogitsLoss()
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

In [ ]:
from cpd_nn.model import CPD_LSTM2, CPD_LSTM
from cpd_nn.model import CPD_GRU

model = CPD_GRU(lstm_model_params['lstm'], lstm_model_params['linear']).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=experiment_params['lr'])
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.97)
loss_fn = torch.nn.HuberLoss()

In [ ]:
if MODEL == 'LSTM':
  config_dict = {
        'model': 'GRU',
        'timeserie-name': dataset_params['data_path'],
        'timeserie-lambda': dataset_params['lambda'],
        'timeserie-alpha': dataset_params['alpha'],
        'timeserie-steps_size': dataset_params['step_size'],
        'timeserie-wins-size': dataset_params['window_size'],
        'timeserie-objective': dataset_params['objective'],
        'timeserie-cooldown': dataset_params['cooldown'],
        'learning_rate': experiment_params['lr'],
        'batch_size': experiment_params['batch_size'],
        'lstm_num_layers': lstm_model_params['lstm']['num_layers'],
        'lstm_dropout': lstm_model_params['lstm']['dropout'],
        'lstm_input': lstm_model_params['lstm']['hidden_dim'],
        'num_epochs': experiment_params['num_epochs'],
        'bidirectional': lstm_model_params['lstm']['is_bidirectional'],
        'scheduling': 'ExponentialLR:gamma=0.98',
        'dataset-round': dataset_params['is_round'],
        'dataset-round-decimals': dataset_params['round_decimals'],
        'dataset-features': dataset_params['feature_type']
    }
elif MODEL == 'MLP':
  config_dict = {
      'model': 'MLP',
      'timeserie-name': dataset_params['data_path'],
      'timeserie-lambda': dataset_params['lambda'],
      'timeserie-alpha': dataset_params['alpha'],
      'timeserie-steps_size': dataset_params['step_size'],
      'learning_rate': experiment_params['lr'],
      'batch_size': experiment_params['batch_size'],
      'n_hidden_layers': mlp_model_params['n_hidden'],
      'hidden_dim': mlp_model_params['hidden_dim'],
      'num_epochs': experiment_params['num_epochs']
  }

In [ ]:
sd.x

array([[ 3.1415927e+00,  2.6782453e-02,  1.0446630e-01, ...,
         9.4243532e-01,  9.6055138e-01, -2.2832155e-01],
       [-9.9147487e-01,  2.9005499e+00, -3.9335381e+01, ...,
         1.9222625e-02, -1.2376984e+00, -1.1721548e+01],
       [ 2.9005499e+00, -3.9335381e+01, -1.4628861e+00, ...,
        -1.2376984e+00, -1.1721548e+01,  4.3258354e-02],
       ...,
       [ 1.6250345e-04,  2.1250527e-04,  2.6575674e-04, ...,
        -2.1117953e-04, -2.6790594e-04, -4.1788130e-04],
       [ 2.1250527e-04,  2.6575674e-04,  2.4656006e-04, ...,
        -2.6790594e-04, -4.1788130e-04, -4.0525236e-04],
       [ 2.6575674e-04,  2.4656006e-04,  3.0184264e-04, ...,
        -4.1788130e-04, -4.0525236e-04, -4.4216478e-04]], dtype=float32)

In [ ]:
wandb.init(
    project='cpd-synth',
    tags=tags,
    notes=notes,
    config = config_dict
)
metrics = train(model, optimizer, loss_fn, train_loader, val_loader, experiment_params['num_epochs'], DEVICE, 
                use_wandb=True, calc_metric=False, scheduler=scheduler, save_weights=True, save_freq=5, save_path='/content/gdrive/MyDrive/cpd/model_weights/')
wandb.finish()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find cpd-synth.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py", line 1146, in init
    wi.setup(kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py", line 289, in setup
    wandb_login._login(
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_login.py", line 298, in _login
    wlogin.prompt_api_key()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_login.py", line 221, in prompt_api_key
    key, status = self._prompt_api_key()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_login.py", line 201, in _prompt_api_key
    key = apikey.prompt_api_key(
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/lib/

Error: ignored

In [ ]:
sd = SynthDatasetTimeToCp(
    ts, 
    dataset_params['min_stability_duration'],
    dataset_params['window_size'],
    dataset_params['train_size'],
    dataset_params['feature_type'],
    dataset_params['balancing'],
    dataset_params['pos_ratio'],
    0
)

  0%|          | 0/3999904 [00:00<?, ?it/s]

In [ ]:
x, y = sd.x.astype(np.float32), sd.y.astype(np.float32)
d = torch.utils.data.TensorDataset(
                torch.tensor(x).unsqueeze(dim=-1),
                torch.tensor(y).unsqueeze(dim=-1)
            )

In [ ]:
from torch.utils.data import Dataset, DataLoader

dloader = DataLoader(d, batch_size=512, shuffle=False)

In [ ]:
scores = []

In [ ]:
from tqdm.notebook import tqdm

model.eval()
for x, y in tqdm(dloader):
  x = x.to(DEVICE)
  y = y.to(DEVICE)
  tmp = model(x)
  tmp = tmp.tolist()
  scores += tmp

  0%|          | 0/7810 [00:00<?, ?it/s]

In [ ]:
print(len(scores))
print(len(dloader.dataset))

3998469
3998469


In [ ]:
import numpy as np

scores = np.array(scores)
with open('/content/gdrive/MyDrive/cpd/synth_logits/100.npy', 'wb') as f:
  np.save(f, scores)